In [147]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt


make mesh

In [148]:
netgen_mesh = unit_square.GenerateMesh(maxh=1.0)
#netgen_dual = unit_square.GenerateMesh(maxh=1.0)
#netgen_primal = unit_square.GenerateMesh(maxh=1.0)
#mesh=Mesh(netgen_mesh) 

# elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
# h = [(2*vol)**(1/2) for vol in elvol]
# print(min(h),max(h))
#help(unit_square)

In [149]:
mesh = Mesh(netgen_mesh)

#mesh.Refine()
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [150]:
vertices = mesh.vertices
n_v = mesh.nv
old_f = mesh.nface
print(n_v)

#mesh.ngmesh.Refine()
# mesh.ngmesh.Update()
# mesh._updateBuffers()

# mesh.Refine()

# mesh.ngmesh.Update()
# mesh.ngmesh.Save("mesh")

# del mesh


# #mesh.ngmesh.Load("mesh.vol.gz")

# mesh = Mesh("mesh.vol.gz")

# mesh._updateBuffers()

mesh.ngmesh.SplitPowellSabin()
# mesh.ngmesh.Update()
# mesh._updateBuffers()
#macromesh = Mesh(mesh.ngmesh.macromesh)

n_f = mesh.nface - old_f
print(n_f)


4
12


In [151]:

#help(mesh)
Draw(mesh)
# for f in mesh.faces:
#     print (f, f.vertices)

data_length = 0

for v in range(n_v):
    vert = mesh.vertices[v]
    print(vert, vert.point)
    print(vert.faces)
    data_length += len(vert.faces)



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

V0 (0.0, 0.0)
(F2, F3, F4, F5)
V1 (1.0, 0.0)
(F6, F7)
V2 (1.0, 1.0)
(F8, F9, F10, F11)
V3 (0.0, 1.0000000000000002)
(F12, F13)


In [152]:
# netgen_primal = unit_square.GenerateMesh(maxh=1.0)
# primal_mesh = Mesh(netgen_primal)
primal_space = H1(mesh, order=1)
u_p = primal_space.TrialFunction()
v_p = primal_space.TestFunction()

# m_p = BilinearForm(primal_space)
# m_p += u_p * v_p *dx
# m_p.Assemble()

dual_space = L2(mesh,order=0)
u_d = dual_space.TrialFunction()
v_d = dual_space.TestFunction()

#help(H1(mesh, order=1))

m_d = BilinearForm(dual_space) #trialspace=primal_space, testspace=dual_space) 

m_d += u_d * v_d * dx

m_d.Assemble()

print(m_d.mat.ToDense())

#help(m.mat)

 0.0833333       0       0       0       0       0       0       0       0       0       0       0
       0 0.0833333       0       0       0       0       0       0       0       0       0       0
       0       0 0.0833333       0       0       0       0       0       0       0       0       0
       0       0       0 0.0833333       0       0       0       0       0       0       0       0
       0       0       0       0 0.0833333       0       0       0       0       0       0       0
       0       0       0       0       0 0.0833333       0       0       0       0       0       0
       0       0       0       0       0       0 0.0833333       0       0       0       0       0
       0       0       0       0       0       0       0 0.0833333       0       0       0       0
       0       0       0       0       0       0       0       0 0.0833333       0       0       0
       0       0       0       0       0       0       0       0       0 0.0833333       0       0
       0  

In [153]:
data = np.empty(data_length)
row_ind = np.empty(data_length, dtype=int)
col_ind = np.empty(data_length, dtype=int)
shape = (n_f,n_v)

i = 0
j = 0

for v in vertices:
    #v = mesh.vertices.__getitem__(i)
    #print(v)
    #vert = mesh.vertices[n_v]
    coeff =  1/len(v.faces)
    dofnr = dual_space.GetDofNrs(v)
    for el in v.elements:
        #print(f)
        dofNr = dual_space.GetDofNrs(el)
        #print(dofNr)
        row_ind[j] = dofNr[0]
        col_ind[j] = v.nr
        data[j] = coeff
        j += 1
    
    i += 1

# print(data)
# print(row_ind)
# print(col_ind)

# sci_mat = sp.sparse.csr_matrix((data, (row_ind, col_ind)),shape)
# print(sci_mat.todense())
# sci_coo = sci_mat.tocoo()

ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, n_f,n_v)
print(ng_mat.T.ToDense())



    0.25    0.25       0       0       0       0    0.25       0       0       0       0    0.25
       0       0     0.5     0.5       0       0       0       0       0       0       0       0
       0       0       0       0    0.25    0.25       0    0.25    0.25       0       0       0
       0       0       0       0       0       0       0       0       0     0.5     0.5       0



In [154]:
#print(m_p.mat.ToDense())

M = ng_mat.T @ m_d.mat @ ng_mat

print(M.ToDense())

 0.0208333       0       0       0
       0 0.0416667       0       0
       0       0 0.0208333       0
       0       0       0 0.0416667



In [155]:


#print(primal_space.CouplingType(0))
# for i in range(primal_space.ndof):
#     primal_space.SetCouplingType(i,COUPLING_TYPE.UNUSED_DOF)

#primal_space.HideAllDofs()

print(primal_space.FreeDofs())
#Draw(macromesh)
#print(macromesh.nv)


0: 11111111111


In [156]:
#Doesnt work bc calc shape wnats floats
# def lincomb(x, y, v, dual_space):

#     func = 0


#     faces = v.faces
#     for el in dual_space.Elements():
#         for f in faces:
#             print(f)
#             if f == el.faces[0]:
#                 func += dual_space.GetFE(el).CalcShape(x=x,y=y)

#     return func

# for f in mesh.faces:
#     print(f,f.vertices)



#help(primal_space)


In [157]:
print(dual_space.ndof)

gfu = GridFunction(dual_space,multidim = dual_space.ndof)

for i in range(dual_space.ndof):
    gfu.vecs[i][i] = 1.0


Draw(gfu, animate = True, min = 0, max = 1.0, deformation = False)

12


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [158]:
dfu = GridFunction(dual_space,multidim = n_v)
#help(ng_mat)



dfu.Set(0)
for i in range(n_v):
    #print(ng_mat.ToDense().NumPy()[:,i])
    dfu.vecs[i][:] = ng_mat.ToDense().NumPy()[:,i]


# for i in range(4):
    #print(dfu.vecs[i][:].FV().NumPy())

Draw(dfu, animate = True, min = 0, max = 0.5, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [159]:
netgen_primal = unit_square.GenerateMesh(maxh=0.5)
primal_mesh = Mesh(netgen_primal)
primal_space = H1(primal_mesh, order=1)

gfu = GridFunction(primal_space,multidim = primal_space.ndof)

for i in range(primal_space.ndof):
    gfu.vecs[i][i] = 1.0


Draw(gfu, animate = True, min = 0, max = 1.0, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene